# Setup

In [13]:
from rtree import index
import pandas as pd
import ast
from dataclasses import dataclass

In [14]:
@dataclass(frozen=True)
class ETA:
    gps_point: tuple
    start_time: str = None
    end_time: str = None

In [15]:
df = pd.read_csv("../stage3/outputs/All_Taxi_Trips.csv")
df['origin_point'] = df['origin_point'].apply(ast.literal_eval)
df['destination_point'] = df['destination_point'].apply(ast.literal_eval)
df['source_timestamp'] = pd.to_datetime(df['source_timestamp'])
df['destination_timestamp'] = pd.to_datetime(df['destination_timestamp'])
df['travel_time'] = df['destination_timestamp'] - df['source_timestamp']
df['travel_time'] = df['travel_time'].apply(lambda x: x.seconds)
df.head()

,trip_id,start_latitude,start_longitude,source_timestamp,origin_point,end_latitude,end_longitude,destination_timestamp,destination_point,cab_id,travel_time
0,1,37.96943,-122.31778,2008-05-17 15:20:33,"(37.96943, -122.31778)",37.79119,-122.40449,2008-05-17 15:40:50,"(37.79119, -122.40449)",0,1217
1,2,37.79505,-122.40479,2008-05-17 15:41:28,"(37.79505, -122.40479)",37.78362,-122.40262,2008-05-17 15:46:48,"(37.78362, -122.40262)",0,320
2,3,37.78363,-122.40261,2008-05-17 15:46:49,"(37.78363, -122.40261)",37.79552,-122.40463,2008-05-17 15:51:49,"(37.79552, -122.40463)",0,300
3,4,37.79593,-122.40495,2008-05-17 15:52:36,"(37.79593, -122.40495)",37.80647,-122.42048,2008-05-17 15:59:31,"(37.80647, -122.42048)",0,415
4,5,37.80648,-122.42048,2008-05-17 15:59:39,"(37.80648, -122.42048)",37.80052,-122.43030,2008-05-17 16:16:11,"(37.80052, -122.4303)",0,992


In [16]:
test_data = pd.read_csv('../stage3/test_student.txt', sep=' ', header=None, index_col=0)

# Create R-Tree

In [19]:
from pyproj import Transformer

# R-Tree index
idx = index.Index()

# Create a transformer to project the latitude and longitude coordinates to the UTM zone 10N coordinate system
transformer = Transformer.from_crs(4326, 32610)

for i, row in df.iterrows():
    sTime = row['source_timestamp']
    dTime = row['destination_timestamp']

    start_point = row['origin_point']
    end_point = row['destination_point']

    start_eta = ETA(start_point, start_time=sTime)
    end_eta = ETA(end_point, end_time=dTime)

    # Project the latitude and longitude coordinates
    x1, y1 = transformer.transform(*start_point)
    x2, y2 = transformer.transform(*end_point)

    idx.insert(i, (x1, y1, x1, y1), start_eta)
    idx.insert(i, (x2, y2, x2, y2), end_eta)

In [20]:
len(idx)

246618

# R-TREE Nearest Neighbor Algorithm

In [48]:
@dataclass
class KNN:
    k: int
    start_point: tuple
    end_point: tuple
    seen_points: dict = None

    def run(self) -> dict:
        self.seen_points = {}

        not_found = True
        while not_found and (self.k < 1000):
            # run for start point
            nearest_points = idx.nearest(coordinates=self.start_point, num_results=self.k, objects=True)

            for point in nearest_points:
                obj = point.object
                # if the point hasn't been seen, add it to the seen_points dict
                if point.id not in self.seen_points:
                    if obj.start_time is not None:
                        self.seen_points[point.id] = {'start_time': obj.start_time}
                    elif obj.end_time is not None:
                        self.seen_points[point.id] = {'end_time': obj.end_time}
                else:
                    if (obj.start_time is not None) and ('start_time' not in self.seen_points[point.id]):
                        self.seen_points[point.id]['start_time'] = obj.start_time
                    elif (obj.end_time is not None) and ('end_time' not in self.seen_points[point.id]):
                        self.seen_points[point.id]['end_time'] = obj.end_time

            # run for end point
            nearest_points = idx.nearest(coordinates=self.end_point, num_results=self.k, objects=True)

            for point in nearest_points:
                obj = point.object

                if (obj.start_time is not None) and (point.id in self.seen_points):
                    if 'start_time' not in self.seen_points[point.id]:
                        self.seen_points[point.id]['start_time'] = obj.start_time
                elif (obj.end_time is not None) and (point.id in self.seen_points):
                    if 'end_time' not in self.seen_points[point.id]:
                        self.seen_points[point.id]['end_time'] = obj.end_time

            important_points = self.seen_points.copy()
            for key, value in self.seen_points.items():
                if len(value) == 1:
                    del important_points[key]

            if len(important_points) == 0:
                self.k += 5
            else:
                not_found = False
                return important_points


In [49]:
from geopy.distance import great_circle

def get_time_diff(time1, time2):
    time1 = pd.to_datetime(time1)
    time2 = pd.to_datetime(time2)
    return abs(time2 - time1).seconds

def get_dist(point1: tuple, point2: tuple) -> float:
    return great_circle(point1, point2).meters

def tie_breaker(test_point: tuple, candidate_ids: dict) -> tuple:
    min_dist = 1000000
    for id in candidate_ids:
        row = df.loc[id]

        start_train_point = row['origin_point']
        end_train_point = row['destination_point']

        start_test_point = test_point[0]
        end_test_point = test_point[1]

        start_dist = get_dist(start_train_point, start_test_point)
        end_dist = get_dist(end_train_point, end_test_point)

        dist = start_dist + end_dist
        if dist < min_dist:
            min_dist = dist
            min_point = id

    return min_point

In [50]:
test_data['travel_time'] = 0
results = test_data.copy()[['travel_time']]
results.head()


,travel_time
0,
0,0
1,0
2,0
3,0
4,0


In [51]:
# run knn algorithm
for i, row in test_data.iterrows():
    start_point = row.loc[2:3]
    end_point = row.loc[5:6]
    test_point = (start_point, end_point)
    x1, y1 = transformer.transform(*start_point)
    x2, y2 = transformer.transform(*end_point)

    knn_obj = KNN(k=5, start_point=(x1, y1, x1, y1), end_point=(x2, y2, x2, y2))
    found_points = knn_obj.run()
    if len(found_points) == 1:
        key, value = found_points.popitem()
        results.at[i, 'travel_time'] = get_time_diff(value['start_time'], value['end_time'])
    elif len(found_points) > 1:
        value = found_points[tie_breaker(test_point, found_points)]
        results.at[i, 'travel_time'] = get_time_diff(value['start_time'], value['end_time'])

In [52]:
results.head()

,travel_time
0,
0,267
1,3067
2,627
3,77
4,15


In [53]:
results.tail()

,travel_time
0,
1896,11
1897,332
1898,60
1899,122
1900,50


In [54]:
results.to_csv('../outputs/eta_knn.csv', index=False)